In [ ]:
from pymongo import MongoClient
import numpy as np
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from collections import defaultdict
import re
from tqdm import tqdm  # Pour la barre de progression
from typing import List, Dict, Set, Any  # Pour le typage
import logging

# Configuration du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuration MongoDB
MONGO_CONFIG = {
    'host': 'localhost',
    'port': 27017,
    'database': 'PFE',
    'collection': 'resumes'
}

In [ ]:

class SkillNormalizer:
    """Classe pour normaliser et regrouper les compétences similaires"""

    def __init__(self):
        self.client = MongoClient(MONGO_CONFIG['host'], MONGO_CONFIG['port'])
        self.db = self.client[MONGO_CONFIG['database']]
        self.collection = self.db[MONGO_CONFIG['collection']]

        # Charger le modèle d'embedding (modèle plus performant pour la production)
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

        # Cache pour les compétences déjà traitées
        self.skill_cache = {}

        # Expressions régulières pour le nettoyage
        self.clean_patterns = [
            (re.compile(r'\(.*?\)'), ''),  # Supprimer les textes entre parenthèses
            (re.compile(r'[^a-zA-Z0-9\s]'), ' '),  # Remplacer les caractères spéciaux
            (re.compile(r'\s+'), ' ')  # Supprimer les espaces multiples
        ]

    def get_all_skills(self) -> List[str]:
        """Récupère toutes les compétences uniques de la collection"""
        skills = set()
        try:
            for doc in self.collection.find({}, {"skills.name": 1}):
                for skill in doc.get("skills", []):
                    if skill.get("name"):
                        cleaned = self._preprocess_skill(skill["name"])
                        if cleaned:
                            skills.add(cleaned)
            return list(skills)
        except Exception as e:
            logger.error(f"Erreur lors de la récupération des compétences: {str(e)}")
            return []

    def _preprocess_skill(self, skill_name: str) -> str:
        """Prétraitement d'une compétence avant clustering"""
        if not skill_name:
            return ""

        # Nettoyage de base
        cleaned = skill_name.lower().strip()

        # Appliquer les regex de nettoyage
        for pattern, repl in self.clean_patterns:
            cleaned = pattern.sub(repl, cleaned)

        return cleaned.strip()

    def cluster_skills(self, skills: List[str]) -> Dict[str, str]:
        """Clusterise les compétences similaires en utilisant des embeddings"""
        if not skills:
            return {}

        logger.info("Génération des embeddings pour le clustering...")
        embeddings = self.model.encode(skills, convert_to_tensor=True)

        logger.info("Clustering des compétences...")
        clustering = DBSCAN(eps=0.4, min_samples=1, metric='cosine').fit(embeddings.cpu())

        # Créer des groupes de compétences similaires
        clusters = defaultdict(list)
        for idx, label in enumerate(clustering.labels_):
            clusters[label].append(skills[idx])

        # Stratégie de sélection du représentant améliorée
        skill_mapping = {}
        for cluster in clusters.values():
            # On choisit le terme le plus courant ou le plus court
            representative = min(cluster, key=lambda x: (len(x.split()), len(x)))
            for skill in cluster:
                skill_mapping[skill] = representative

        return skill_mapping

    def process_document(self, doc: Dict[str, Any], skill_mapping: Dict[str, str]) -> Dict[str, Any]:
        """Traite un document avec le mapping dynamique"""
        try:
            # Traitement des compétences
            if 'skills' in doc:
                cleaned_skills = []
                skill_scores = defaultdict(float)

                for skill in doc['skills']:
                    if not skill.get('name'):
                        continue

                    cleaned_name = self._preprocess_skill(skill['name'])
                    if not cleaned_name:
                        continue

                    # Utiliser le mapping ou garder le nom original si non mappé
                    normalized_name = skill_mapping.get(cleaned_name, cleaned_name).title()

                    # Garder le score le plus élevé pour chaque compétence normalisée
                    skill_scores[normalized_name] = max(
                        skill_scores[normalized_name],
                        float(skill.get('score', 0))
                    )

                # Recréer la liste des compétences
                doc['skills'] = [
                    {'name': name, 'score': score}
                    for name, score in skill_scores.items()
                    if name  # Exclure les noms vides
                ]


            # Conversion des années d'expérience en entier
            if 'years_of_experience' in doc:
                try:
                    # Si c'est déjà un nombre, on le convertit en int directement
                    if isinstance(doc['years_of_experience'], (int, float)):
                        doc['years_of_experience'] = int(doc['years_of_experience'])
                    # Si c'est une chaîne, on essaie d'extraire le nombre
                    elif isinstance(doc['years_of_experience'], str):
                        # Extraire les chiffres de la chaîne
                        years_str = re.search(r'\d+', doc['years_of_experience'])
                        if years_str:
                            doc['years_of_experience'] = int(years_str.group())
                        else:
                            doc['years_of_experience'] = 0
                except (ValueError, TypeError) as e:
                    logger.warning(f"Impossible de convertir years_of_experience pour le document {doc.get('_id')}: {str(e)}")
                    doc['years_of_experience'] = 0

            return doc

        except Exception as e:
            logger.error(f"Erreur lors du traitement du document {doc.get('_id')}: {str(e)}")
            return doc

    def run(self):
        """Exécute le processus complet de normalisation"""
        try:
            logger.info("Début du processus de normalisation des compétences...")

            # Étape 1: Récupérer et clusteriser les compétences
            all_skills = self.get_all_skills()
            logger.info(f"Nombre de compétences uniques avant clustering: {len(all_skills)}")

            skill_mapping = self.cluster_skills(all_skills)
            unique_skills = set(skill_mapping.values())
            logger.info(f"Nombre de compétences uniques après clustering: {len(unique_skills)}")

            # Étape 2: Traiter tous les documents
            total_docs = self.collection.count_documents({})
            logger.info(f"Traitement de {total_docs} documents...")

            for doc in tqdm(self.collection.find(), total=total_docs):
                try:
                    cleaned_doc = self.process_document(doc, skill_mapping)

                    # Mettre à jour le document
                    update_data = {
                        'skills': cleaned_doc.get('skills', [])
                    }

                    # Ajouter years_of_experience seulement s'il est présent dans le doc nettoyé
                    if 'years_of_experience' in cleaned_doc:
                        update_data['years_of_experience'] = cleaned_doc['years_of_experience']

                    self.collection.update_one(
                        {'_id': doc['_id']},
                        {'$set': update_data}
                    )

                except Exception as e:
                    logger.error(f"Erreur avec le document {doc.get('_id')}: {str(e)}")
                    continue

            logger.info("Nettoyage intelligent terminé avec succès!")

        finally:
            self.client.close()


In [6]:
if __name__ == "__main__":
    normalizer = SkillNormalizer()
    normalizer.run()

2025-06-18 12:31:31,076 - INFO - Use pytorch device_name: cpu
2025-06-18 12:31:31,078 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-06-18 12:31:33,027 - INFO - Début du processus de normalisation des compétences...
2025-06-18 12:31:33,037 - INFO - Nombre de compétences uniques avant clustering: 230
2025-06-18 12:31:33,039 - INFO - Génération des embeddings pour le clustering...


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

2025-06-18 12:31:33,947 - INFO - Clustering des compétences...
2025-06-18 12:31:33,959 - INFO - Nombre de compétences uniques après clustering: 230
2025-06-18 12:31:33,963 - INFO - Traitement de 43 documents...
100%|██████████| 43/43 [00:00<00:00, 207.73it/s]
2025-06-18 12:31:34,184 - INFO - Nettoyage intelligent terminé avec succès!
